# 3. Content Based filtering

In [1]:
!pip install gensim
!pip install jieba
!pip install underthesea

     |████████████████████████████████| 7.6 MB 4.4 MB/s 
     |████████████████████████████████| 235 kB 65.1 MB/s 
     |████████████████████████████████| 581 kB 50.8 MB/s 
     |████████████████████████████████| 743 kB 60.6 MB/s 


In [2]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

%cd '/content/gdrive/My Drive/LDS0_K273_ONLINE_DoThiPhuong/Topic_2/'

Mounted at /content/gdrive
/content/gdrive/My Drive/LDS0_K273_ONLINE_DoThiPhuong/Topic_2


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
warnings.filterwarnings('ignore')

In [4]:
# đọc dữ liệu
products = pd.read_csv('Products.csv')
reviews = pd.read_csv('Reviews.csv')

In [5]:
products.head(2)

,item_id,name,description
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời g...
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,THÔNG TIN CHI TIẾT\nDung lượng pin 2000mah\nTh...


In [6]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4373 entries, 0 to 4372
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   item_id      4373 non-null   int64 
 1   name         4373 non-null   object
 2   description  4370 non-null   object
dtypes: int64(1), object(2)
memory usage: 102.6+ KB


In [7]:
products.shape

(4373, 3)

In [8]:
products = products[products['name'].notnull()]

In [9]:
products['name_description'] = products.name + products.description

In [10]:
products = products[products['name_description'].notnull()]

In [11]:
products['name_description_pre'] = products['name_description'].apply(lambda x: word_tokenize(x, format = 'text'))

In [12]:
type(products)

pandas.core.frame.DataFrame

In [13]:
products.shape

(4370, 5)

In [14]:
products.head(2)

,item_id,name,description,name_description,name_description_pre
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời g...,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,Tai_nghe Bluetooth_Inpods 12 - Cảm_biến vân ta...
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,THÔNG TIN CHI TIẾT\nDung lượng pin 2000mah\nTh...,Tai nghe bluetooth không dây F9 True wireless ...,Tai_nghe bluetooth không dây F9_True wireless ...


In [15]:
products = products.reset_index()

## 3.2 Áp dụng Gensim

In [16]:
from gensim import corpora, models, similarities
import jieba
import re

In [17]:
# Tokenize the sentences into words
intro_products =  [[text for text in x.split()] for x in products.name_description_pre]

In [18]:
len(intro_products)

4370

In [19]:
intro_products[:1]

[['Tai_nghe',
  'Bluetooth_Inpods',
  '12',
  '-',
  'Cảm_biến',
  'vân',
  'tay',
  ',',
  'chống',
  'nước',
  ',',
  'màu_sắc',
  'đa_dạng',
  '-',
  '5',
  'màu_sắc',
  'lựa',
  'chọnTHÔNG',
  'TIN',
  'CHI_TIẾT',
  'Dung_lượng',
  'pin',
  '300',
  'Thời_gian',
  'pin',
  '-',
  'Thời_gian',
  'nghe',
  'nhạc',
  'liên_tục',
  'từ',
  '2.5',
  '-',
  '4',
  'h',
  '-',
  'Thời_gian',
  'sạc',
  'đầy',
  'chỉ',
  'khoảng',
  '60',
  'p',
  '-',
  'Thời_gian',
  'chờ',
  'lên',
  'tới',
  '140',
  'giờ',
  'Bluetooth',
  '5',
  'Thương_hiệu',
  'OEM',
  'Xuất_xứ',
  'thương_hiệu',
  'Trung_Quốc',
  'Độ',
  'nhạy_cảm_biến',
  'vân',
  'tay',
  'Model',
  'i12',
  'Loại',
  'Jack',
  'cắm',
  'USB_Cable',
  'Trọng_lượng',
  '300',
  'g',
  'Thời_gian',
  'sử_dụng',
  '-',
  'Thời_gian',
  'nghe',
  'nhạc',
  'liên_tục',
  'từ',
  '2.5',
  '-',
  '4',
  'h',
  'SKU',
  '4096608751631',
  'MÔ_TẢ',
  'SẢN_PHẨM',
  'INPOD_12',
  'là',
  'phiên_bản',
  'nâng_cấp',
  'mới',
  'nhất',
  ',',

In [20]:
STOP_WORD_FILE = 'vietnamese-stopwords.txt'

In [21]:
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
  stop_words = file.read()

stop_words = stop_words.split('\n')

In [22]:
# remove some special elements in texts
intro_products_re = [[re.sub('[0-9]+','', e) for e in text] for text in intro_products] # số
intro_products_re = [[t.lower() for t in text if not t in ['', ' ', ',', '.', '...', '-',':', ';', '?', '%', '(', ')', '+', '/']] for text in  intro_products_re] # ký tự đặc biệt
intro_products_re = [[t for t in text if not t in stop_words] for text in intro_products_re] # stopword

In [23]:
intro_products_re[:1]

[['tai_nghe',
  'bluetooth_inpods',
  'cảm_biến',
  'vân',
  'tay',
  'chống',
  'nước',
  'màu_sắc',
  'đa_dạng',
  'màu_sắc',
  'lựa',
  'chọnthông',
  'chi_tiết',
  'dung_lượng',
  'pin',
  'pin',
  'nhạc',
  'liên_tục',
  'h',
  'sạc',
  'p',
  'chờ',
  'bluetooth',
  'thương_hiệu',
  'oem',
  'xuất_xứ',
  'thương_hiệu',
  'trung_quốc',
  'độ',
  'nhạy_cảm_biến',
  'vân',
  'tay',
  'model',
  'jack',
  'cắm',
  'usb_cable',
  'trọng_lượng',
  'g',
  'nhạc',
  'liên_tục',
  'h',
  'sku',
  'mô_tả',
  'sản_phẩm',
  'inpod_',
  'phiên_bản',
  'nâng_cấp',
  'tai_nghe',
  'bluetooth_.',
  'thiết_kế',
  'tỉ_lệ',
  'chuẩn',
  'tai',
  'airpod',
  'hãng',
  'lược_bỏ',
  'nút',
  'bấm',
  'thân',
  'tai',
  'thay',
  'nút',
  'cảm_ứng',
  'dễ_dàng',
  'thuận_tiện',
  'thao_tác',
  'nhạc',
  'dễ_dàng',
  'chạm',
  'bluetooth_.',
  'kết_nối',
  'vô_cùng',
  'ổn_định',
  'bluetooth_.',
  'kết_nối',
  'vô_cùng',
  'ổn_định',
  'tai',
  'kết_nối',
  'dock',
  'sạc',
  'chất',
  'âm',
  'thời_lư

In [24]:
intro_products_re = [[t for t in text if not t in ['p', 'g', 'vô_cùng', 'v', '._v', 'h', 'tiki','luật', 'hiện_hành', 'tuỳ', 'sản_phẩm', 'phương_thức', 'địa_chỉ', 'giao', 'phát_sinh', 'chi_phí', 'phí', 'vận_chuyển', 'phụ_phí', 'hàng', 'cồng_kềnh']] for text in  intro_products_re]

In [25]:
intro_products_re[:1]

[['tai_nghe',
  'bluetooth_inpods',
  'cảm_biến',
  'vân',
  'tay',
  'chống',
  'nước',
  'màu_sắc',
  'đa_dạng',
  'màu_sắc',
  'lựa',
  'chọnthông',
  'chi_tiết',
  'dung_lượng',
  'pin',
  'pin',
  'nhạc',
  'liên_tục',
  'sạc',
  'chờ',
  'bluetooth',
  'thương_hiệu',
  'oem',
  'xuất_xứ',
  'thương_hiệu',
  'trung_quốc',
  'độ',
  'nhạy_cảm_biến',
  'vân',
  'tay',
  'model',
  'jack',
  'cắm',
  'usb_cable',
  'trọng_lượng',
  'nhạc',
  'liên_tục',
  'sku',
  'mô_tả',
  'inpod_',
  'phiên_bản',
  'nâng_cấp',
  'tai_nghe',
  'bluetooth_.',
  'thiết_kế',
  'tỉ_lệ',
  'chuẩn',
  'tai',
  'airpod',
  'hãng',
  'lược_bỏ',
  'nút',
  'bấm',
  'thân',
  'tai',
  'thay',
  'nút',
  'cảm_ứng',
  'dễ_dàng',
  'thuận_tiện',
  'thao_tác',
  'nhạc',
  'dễ_dàng',
  'chạm',
  'bluetooth_.',
  'kết_nối',
  'ổn_định',
  'bluetooth_.',
  'kết_nối',
  'ổn_định',
  'tai',
  'kết_nối',
  'dock',
  'sạc',
  'chất',
  'âm',
  'thời_lượng',
  'pin',
  'cải_thiện',
  'tối_ưu',
  'dock',
  'sạc',
  'tiện

In [26]:
dictionary = corpora.Dictionary(intro_products_re)

In [27]:
dictionary.token2id

{'airpod': 0,
 'apple': 1,
 'bao_gồm': 2,
 'bluetooth': 3,
 'bluetooth_.': 4,
 'bluetooth_inpods': 5,
 'bấm': 6,
 'chi_tiết': 7,
 'chuẩn': 8,
 'chạm': 9,
 'chất': 10,
 'chọnthông': 11,
 'chống': 12,
 'chờ': 13,
 'cải_thiện': 14,
 'cảm_biến': 15,
 'cảm_ứng': 16,
 'cắm': 17,
 'dock': 18,
 'dung_lượng': 19,
 'dễ_dàng': 20,
 'giá': 21,
 'huawei': 22,
 'hãng': 23,
 'inpod_': 24,
 'jack': 25,
 'kết_nối': 26,
 'lenovo': 27,
 'liên_tục': 28,
 'lược_bỏ': 29,
 'lựa': 30,
 'model': 31,
 'màu_sắc': 32,
 'mô_tả': 33,
 'nhạc': 34,
 'nhạy_cảm_biến': 35,
 'nâng_cấp': 36,
 'nút': 37,
 'nước': 38,
 'oem': 39,
 'oppo': 40,
 'phiên_bản': 41,
 'pin': 42,
 'samsung': 43,
 'sku': 44,
 'sạc': 45,
 'tablet': 46,
 'tai': 47,
 'tai_nghe': 48,
 'tay': 49,
 'thao_tác': 50,
 'thay': 51,
 'thiết_bị': 52,
 'thiết_kế': 53,
 'thuận_tiện': 54,
 'thuế': 55,
 'thân': 56,
 'thương_hiệu': 57,
 'thời_lượng': 58,
 'tiện_lợi': 59,
 'trung_quốc': 60,
 'trọng_lượng': 61,
 'tương_thích': 62,
 'tỉ_lệ': 63,
 'tối_ưu': 64,
 'usb_cab

In [28]:
len(dictionary.token2id)

40879

In [29]:
feature_cnt = len(dictionary.token2id)

In [30]:
feature_cnt

40879

In [31]:
corpus = [dictionary.doc2bow(text) for text in intro_products_re]

In [32]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 2),
 (4, 3),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 2),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 2),
 (19, 1),
 (20, 2),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 3),
 (27, 1),
 (28, 3),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 2),
 (33, 1),
 (34, 4),
 (35, 1),
 (36, 1),
 (37, 2),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 4),
 (43, 1),
 (44, 1),
 (45, 5),
 (46, 1),
 (47, 3),
 (48, 3),
 (49, 2),
 (50, 1),
 (51, 1),
 (52, 2),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 2),
 (58, 1),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 2),
 (68, 1),
 (69, 1),
 (70, 1),
 (71, 1),
 (72, 1),
 (73, 2)]

In [33]:
tfidf = models.TfidfModel(corpus)
# tính toán sự tương đồng trong ma trận thưa thớt
index = similarities.SparseMatrixSimilarity(tfidf[corpus],
                                            num_features = feature_cnt)

In [34]:
# Khi user chọn 1 sp: product_id = 48102821
product_ID = 48102821
product = products[products.item_id == product_ID].head(1)

In [35]:
product[['index', 'item_id', 'name_description_pre']]

,index,item_id,name_description_pre
0,0,48102821,Tai_nghe Bluetooth_Inpods 12 - Cảm_biến vân ta...


In [36]:
# sp đang xem
name_description_pre = product['name_description_pre'].to_string(index = False)

In [37]:
name_description_pre

'Tai_nghe Bluetooth_Inpods 12 - Cảm_biến vân tay...'

Đề xuất cho sp đang xem

In [38]:
def recommender(view_product, dictionary, tfidf, index):
  view_product = view_product.lower().split()
  kw_vector = dictionary.doc2bow(view_product)
  print("View product's vector:")
  print(kw_vector)
  # similarity calculation
  sim = index[tfidf[kw_vector]]

  # print result 
  list_id = []
  list_score = []
  for i in range(len(sim)):
    list_id.append(i)
    list_score.append(sim[i])
  
  df_result = pd.DataFrame({
      'id': list_id,
      'score': list_score
  })
  # 5 highest scores
  five_highest_scores = df_result.sort_values(by = 'score', ascending = False).head(6)
  print('5 highest scores:')
  print(five_highest_scores)
  print('Ind to list:')
  id2list = list(five_highest_scores['id'])
  print(id2list)

  products_find  = products[products.index.isin(id2list)]
  results = products_find[['index', 'item_id', 'name']]
  results = pd.concat([results, five_highest_scores], axis = 1).sort_values(by = 'score', ascending = False)
  return results

In [39]:
results = recommender(name_description_pre, dictionary, tfidf, index)

View product's vector:
[(5, 1), (15, 1), (48, 1), (67, 1)]
5 highest scores:
      id     score
0      0  0.286694
75    75  0.226480
23    23  0.144560
719  719  0.129248
587  587  0.126589
245  245  0.125531
Ind to list:
[0, 75, 23, 719, 587, 245]


In [40]:
# đề xuất 5 sp tương tự với sp đang xem
# xóa bỏ sp đang xem trong danh sách đề xuất
results = results[results.item_id != product_ID]
results

,index,item_id,name,id,score
75,75,35607267,Tai nghe Bluetooth Inpods 12 Thời trang,75,0.226480
23,23,35373097,Tai Nghe Bluetooth True Wireless AMOI F9 5.0 C...,23,0.144560
719,719,48273751,Tai nghe Blutooth 5.0 kiêm dock sạc dự phòng- ...,719,0.129248
587,587,79965318,Tai nghe Bluetooth Lanith – Tai Nghe Không Dây...,587,0.126589
245,245,58291928,Tai Nghe True Wireless Earbuds QCY T7 Bluetoot...,245,0.125531


In [41]:
def recommendation(product_ID):
  product = products[products.item_id == product_ID].head(1)
  name_description_pre = product['name_description_pre'].to_string(index = False)
  results = recommender(name_description_pre, dictionary, tfidf, index)
  results = results[results.item_id != product_ID]
  return results

In [42]:
results = recommendation(48102821)
results

View product's vector:
[(5, 1), (15, 1), (48, 1), (67, 1)]
5 highest scores:
      id     score
0      0  0.286694
75    75  0.226480
23    23  0.144560
719  719  0.129248
587  587  0.126589
245  245  0.125531
Ind to list:
[0, 75, 23, 719, 587, 245]


,index,item_id,name,id,score
75,75,35607267,Tai nghe Bluetooth Inpods 12 Thời trang,75,0.226480
23,23,35373097,Tai Nghe Bluetooth True Wireless AMOI F9 5.0 C...,23,0.144560
719,719,48273751,Tai nghe Blutooth 5.0 kiêm dock sạc dự phòng- ...,719,0.129248
587,587,79965318,Tai nghe Bluetooth Lanith – Tai Nghe Không Dây...,587,0.126589
245,245,58291928,Tai Nghe True Wireless Earbuds QCY T7 Bluetoot...,245,0.125531


In [43]:
results = recommendation(10001355)
results

View product's vector:
[(297, 1), (309, 1), (21542, 1)]
5 highest scores:
        id     score
2314  2314  0.484820
2505  2505  0.467288
2413  2413  0.460508
2473  2473  0.453879
2658  2658  0.448651
2662  2662  0.447897
Ind to list:
[2314, 2505, 2413, 2473, 2658, 2662]


,index,item_id,name,id,score
2314,2315,10001369,RAM Laptop Samsung 8GB DDR4 2666MHz SODIMM - H...,2314,0.484820
2505,2507,10001357,RAM Laptop Hynix 8GB DDR4 2400MHz SODIMM - Hàn...,2505,0.467288
2413,2414,10001373,RAM Laptop Samsung 16GB DDR4 2666MHz SODIMM - ...,2413,0.460508
2473,2474,10001377,RAM Laptop Hynix 8GB DDR4 2666MHz SODIMM - Hàn...,2473,0.453879
2662,2664,10001334,RAM Laptop Hynix 8GB DDR4 2133MHz SODIMM - Hàn...,2662,0.447897


In [44]:
products[products.item_id == 10001355].name

2658    RAM Laptop Hynix 4GB DDR4 2400MHz SODIMM - Hàn...
Name: name, dtype: object

## Nhận xét:
- Nhìn chung 2 thuật toán cho kết quả đề xuất tương đối tốt.
- Gensim tốn ít không gian lưu trữ hơn.